<a href="https://colab.research.google.com/github/Samir-atra/BDSD_Minor_Project/blob/main/MediaPipe_Stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and install packages for colab
# !python -m pip install mediapipe
# !wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
# !wget -q -O image.png https://storage.googleapis.com/mediapipe-assets/business-person.png

# Imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pathlib
# from google.colab.patches import cv2_imshow
import csv
import pandas as pd
import tensorflow as tf
import keras_tuner
import math
import time
import os

In [ ]:
# MediaPipe visualizations


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()


In [ ]:
# a function to write dta into csv files
def csv_writer(filename, fields, data):

    csvfile = filename
    fields = fields
    with open(csvfile, mode="a") as first:
        csvwriter = csv.writer(first)
        csvwriter.writerow(fields)
        csvwriter.writerows(data)

        return True

In [ ]:
# download and upload images from and into colab

# download
img = cv2.imread("images.jpg")
cv2_imshow(img)

# upload images to colab
from google.colab import files
uploaded = files.upload()

for filename in uploaded:
  content = uploaded[filename]
  with open(filename, 'wb') as f:
    f.write(content)

if len(uploaded.keys()):
  IMAGE_FILE = next(iter(uploaded))
  print('Uploaded file:', IMAGE_FILE)

In [ ]:
#inferencing and visualizing

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.png")

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)
# print(detection_result.facial_transformation_matrixes)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
plt.imshow(annotated_image)
plt.show()

In [ ]:
# plot bar graphs for specific image blendshapes

plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
# print(detection_result.face_blendshapes[0])
print(detection_result)

In [ ]:
# dataset analysis/calculations

categories_counts = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}
skipped = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}


with open("fer2013.csv", mode= "r") as data:                    # open the dataset file
  csvFile = csv.reader(data)
  next(csvFile)
  for lines in csvFile:                                         # loop throught the training instances
    if lines[0] in categories_c:
      categories_counts[lines[0]] = categories_counts[lines[0]] + 1
    image = np.array(str(lines[1]).split(' ')).reshape(48, 48, 1).astype(np.uint8)     # build the image from a list of numbers
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
    detection_result = detector.detect(rgb_frame)                                       # detect the face in the image
    # check if the image get recognized or skipped by mediapipe
    if detection_result.face_blendshapes == []:                                         # check if mediapipe is able to detect a face in the image
        skipped[lines[0]] = skdfivjipped[lines[0]] + 1
    else:
        # img = plt.imshow(image)
        # plt.show()
        continue
print(categories_counts)
print(skipped)

In [ ]:
# # Processing, cleaning and visualizing the fer2023 dataset  

fullset = []
Training_set = []
validation_set = []
test_set = []
class_counter = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}


with open("training_set_full.csv", mode= "r") as data:
  csvFile = csv.reader(data)
  next(csvFile)
  # iterate over the dataset and append the images to a list, for limited numbers according to it's classes, to creat the training set
  for lines in csvFile:
      if lines[0] == "0" and class_counter['0'] < 1500:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '1'
        fullset.append(lines)
      elif lines[0] == "1" and class_counter['1'] < 1500:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '1'
        fullset.append(lines)
      elif lines[0] == '2' and class_counter['2'] < 1500:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '1'
        fullset.append(lines) 
      elif lines[0] == '3' and class_counter['3'] < 4000:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '0'
        fullset.append(lines) 
      elif lines[0] == '4' and class_counter['4'] < 4000:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '2'
        fullset.append(lines) 
      elif lines[0] == '5' and class_counter['5'] < 1500:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '1'
        fullset.append(lines) 
      elif lines[0] == '6' and class_counter['6'] < 1500:
        class_counter[lines[0]] = class_counter[lines[0]] + 1
        lines[0] = '1'
        fullset.append(lines) 

print(class_counter)
# plot the required image with annotations
plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
annotated_image = draw_landmarks_on_image(rgb_frame.numpy_view(), detection_result)
cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))



In [ ]:
# cleaning the dataset 

training_set = []
validation_set = []
test_set = []

# creat lists for dataset splits
for i in fullset:
    if i[2] == 'Training':
        training_set.append(i)
    elif i[2] == 'PublicTest':
        validation_set.append(i)
    elif i[2] == 'PrivateTest':
        test_set.append(i)

training_set_hus = []
validation_set_hus = []
test_set_hus = []

# append the understandable images of the original dataset splits to new lists
for lines in training_set:
      image = np.array(lines[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
      image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    #   plt.imshow(image)
    #   plt.show()
      frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
      detection_result = detector.detect(frame)
      if detection_result.face_blendshapes == []:
        continue
      else:
        training_set_hus.append(lines)

for val_line in validation_set:
      imagee = np.array(val_line[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
      imagee = cv2.cvtColor(imagee, cv2.COLOR_GRAY2RGB)
    #   plt.imshow(image)
    #   plt.show()
      framee = mp.Image(image_format=mp.ImageFormat.SRGB,data=imagee)
      detection_result = detector.detect(framee)
      if detection_result.face_blendshapes == []:
        continue
      else:
        validation_set_hus.append(val_line)

for test_lines in test_set:
      imageee = np.array(test_lines[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
      imageee = cv2.cvtColor(imageee, cv2.COLOR_GRAY2RGB)
    #   plt.imshow(image)
    #   plt.show()
      frameee = mp.Image(image_format=mp.ImageFormat.SRGB,data=imageee)
      detection_result = detector.detect(frameee)
      if detection_result.face_blendshapes == []:
        continue
      else:
        test_set_hus.append(test_lines)

# create files for the new dataset splits for the created lists
fields = ["emotion", "pixels", "Usage"]
csv_writer("training_set_full.csv", fields, training_set_hus)
csv_writer("validation_set_full.csv", fields, validation_set_hus)
csv_writer("test_set_full.csv", fields, test_set_hus)


In [ ]:
# choosing relevant blendshapes for the happy, sad dataset and the happy, sad, neutral dataset 


# create a dictionary with numbers from 0 to 52 
blend_shapes = dict()
for i in range(0,52):
    blend_shapes[str(i)] = 0

print(blend_shapes)
# sad = 0
# happy = 0
counter = 0

  # find which blendshapes are most relevant to happiness and sadness by passing them on a 0.4 threshold 
for i in range(len(training_set_hus)):
    image = np.array(training_set_hus[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)    # build images from pixels lists
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
    detection_result = detector.detect(frame)
    if detection_result.face_blendshapes == []:
        continue
    else:
        counter += 1              
    if counter %500 == 0:
        time.sleep(5)                 # sleep 5 seconds to avoid processor overload
    for i in detection_result.face_blendshapes[0]:           # compare each blendshape in for the current image with the threshold
        if i.score > 0.4:
            blend_shapes[str(i.index)] = blend_shapes[str(i.index)] + 1           # edit the counts dictionary
print(blend_shapes)

# from the resulting dictionary, manually find the modst relevent blendshapes and insert them in a list

blends_to_print = ['1', '2', '3', '4', '5', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '25', '34', '35', '38', '44', '45', '46', '47', '48', '49', 'emotion'] # list of blendshapes indices that are most relevent


In [ ]:
# indexing the testset

test_set_hus = []

with open("test_set_full.csv", mode= "r") as data:
  csvFile = csv.reader(data)
  next(csvFile)
  for lines in csvFile:
      imageee = np.array(lines[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
      imageee = cv2.cvtColor(imageee, cv2.COLOR_GRAY2RGB)
    #   plt.imshow(image)
    #   plt.show()
      frameee = mp.Image(image_format=mp.ImageFormat.SRGB,data=imageee)
      detection_result = detector.detect(frameee)
      if detection_result.face_blendshapes == []:
        continue
      else:
        test_set_hus.append(lines)
        
nums = np.array([i for i in range(0,1648)])               # create a list of numbers to be appended as indices
nums = np.reshape(nums, (1648,1))
test_set_hus = np.array(test_set_hus)
test_set_hus= np.delete(test_set_hus,2,1)                 # delete the third column which contains the name of the split
test_set_hus = np.hstack((test_set_hus, nums))            # add the indices column
fields = ["emotion", "pixels", "Index"]
csv_writer("test_set_full_index.csv", fields, test_set_hus)          # creat an indexed dataset


In [ ]:
# Augmenting the training set images


augmented_training_set = []
training_images = []
training_labels = []

# create an image list and a labels list for the training dataset
for i in range(math.floor(len(training_set_hus))):
    image = np.array(training_set_hus[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    training_images.append(image)
    training_labels.append(int(training_set_hus[i][0]))


# rescaling and augmenting images models
rescaling1 = tf.keras.Sequential([ 
  tf.keras.layers.Rescaling(1./255)                         # scale down the images pixel values
])

rescaling2 = tf.keras.Sequential([                          # scale up the pixel values
  tf.keras.layers.Rescaling(1.*255)
])

augment = tf.keras.Sequential([                             # augment by random flipping and random rotation
  tf.keras.layers.RandomFlip("horizontal"),
  tf.keras.layers.RandomRotation(0.1)
])

for ele in range(len(training_images)):
# scale down the image and augment
  img = training_images[ele]
  label = training_labels[ele]
  image = rescaling1(img)
  aug_image = augment(image)
#   scale up the image cast to an integer and transform into a numpy array for it to be understood by mediapipe
  aug_image = rescaling2(aug_image)
  aug_image = tf.cast(aug_image, tf.uint8)
  aug_image = np.array(aug_image)
  flatten_image = aug_image.flatten()                                            # flatten the augmented image, to be used in creating the csv file
  flat_aug_image = [flatten_image[i] for i in range(0,len(flatten_image),3)]
  # flattt = np.reshape(flat_aug_image,(48,48))
  # plt.imshow(flattt)
  # plt.show()
  frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=aug_image)
  detection_result = detector.detect(frame)
  if detection_result.face_blendshapes == []:
    continue
  else:
    element = [training_labels[ele]]
    for i in flat_aug_image:
      element.append(i)
    augmented_training_set.append([element[0],str(element[1:]).replace(',',"").replace('[','').replace(']',''),'Training'])

for images in augmented_training_set:
  image = np.array(images[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  plt.imshow(image)
  plt.show()

csv_writer("training_set_full.csv", ['emotion','pixels'], augmented_training_set)


In [ ]:
# if-else classification 😊

# if detection_result.face_blendshapes[0][44].score > 0.5 and detection_result.face_blendshapes[0][45].score > 0.5 :
#   print("Happy")
# elif detection_result.face_blendshapes[0][42].score > 0.5:
#   print("Sad")


In [ ]:
# creating blendshapes dataset        (styled)

set = []
images = []
labels = []
full_set = []
indices = []

with open("validation_set_full.csv", mode= "r") as data:            # load the dataset that will be processed
  csvFile = csv.reader(data)
  next(csvFile)
  for lines in csvFile:
    set.append(lines)
  for i in range(len(set)):    
    image = np.array(set[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    images.append(image)                               # create a list of the images
    labels.append(int(set[i][0]))                      # create a list of the labels
    # indices.append(int(set[i][2]))
    
    
arr = np.zeros((len(images), 36))                    # create an array with the size of dataset and number of blendshapes to be used 

# arr[0,:]= blendS_to_print
for ele in range(len(images)-1):
  img = images[ele]
  label = labels[ele]
  # img_index = indices[ele]
  frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=img)
  detection_result = detector.detect(frame)
  cat_counter = 0
  for category in detection_result.face_blendshapes[0]:
    if str(category.index) in blends_to_print:                # if the blendshape from the detector is in the blends_to_pront list then include it in the array
      arr[ele, cat_counter] = category.score
      cat_counter += 1
    else:
       continue
  arr[ele, 34] = label
  # arr[ele, 35] = img_index

     
# fields = ["emotion", "pixels", "Index"]
csv_writer("blends_val_all_emotion.csv",'beedoo', arr)             # create a file from the array


In [ ]:
# load data for LSTM model         ()

traindata = []
blend_set = []
labels_set = []
class1=[]
class2=[]
class3=[]
with open("blends_train_full_set.csv", mode= "r") as data:
  csvFile = csv.reader(data)
  next(csvFile)
  for line in csvFile:
    traindata.append(line[:])
  np.random.shuffle(traindata)
  for lines in traindata:
      blend_set.append(lines[0:52])
      labels_set.append(lines[52])
blends_set = np.array(blend_set, dtype=np.float64)        # create an array of the blendshapes
labels_set = np.array(labels_set, dtype=np.float64)       # create an array of the labels

#Reshaping Array
X_train = np.reshape(blends_set, (22515, 52,1))
Y_train = np.reshape(labels_set, (22515,1)).astype('int')
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=3)       # encode the labels as one-hot

# create classes lists for it to be produced as datasets
for ins in traindata:
  if float(ins[52]) == 0:
    class1.append(ins)
  elif float(ins[52]) == 1:
    class2.append(ins)
  elif float(ins[52]) == 2:
    class3.append(ins)
    
csv_writer("class1.csv",'beedoo', class1)
csv_writer("class2.csv",'beedoo', class2)
csv_writer("class3.csv",'beedoo', class3)

# visualizations for the features from the classes and 
plt.figure(figsize = [40,40])
y = np.linspace(0,1999,1999)
for j in range(1,52):
  plt.subplot(10,6,j)                                         # create a subplot with the number of features
  class1_slice = [float(i[j]) for i in blends_set[1:2000]]
# class2_slice = [float(i[25]) for i in class2[1:200]]
  plt.title(f"{j}")
  plt.scatter(y,class1_slice,color='blue')
plt.scatter(y,class2_slice, color='red')
print(class1[1:100])
corr_mat = np.corrcoef(class1_slice, class2_slice)            # find the correlation matrix
print(corr_mat)

# load validation data
valdata = []
val_blend_set = []
val_labels_set = []
with open("blends_val_all_emotion.csv", mode= "r") as val_data:
  csvFile = csv.reader(val_data)
  next(csvFile)
  for line in csvFile:
    valdata.append(line[:])
  np.random.shuffle(valdata)
  for lines in valdata:
      val_blend_set.append(lines[0:34])
      val_labels_set.append(lines[34])
val_blend_set = np.array(val_blend_set, dtype=np.float64)
val_labels_set = np.array(val_labels_set, dtype=np.float64)
X_val = np.reshape(val_blend_set, (1657, 34,1))
y_val = np.reshape(val_labels_set, (1657,1)).astype('int')
y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)



In [ ]:
# data normalization

mean = tf.math.reduce_mean(X_train,axis=0)            # find the mean for the dataset
stddev = tf.math.reduce_std(X_train, axis=0)          # find the standard deviation
mean = np.array(mean).T
stddev = np.array(stddev).T
csv_writer("mean_and_std.csv",'beedoo', mean)
csv_writer("mean_and_std.csv",'beedoo', stddev)

# normalize data
norm = tf.keras.layers.Normalization(axis=1)
norm.adapt(X_train)
print(X_train[0])
XX_train = norm(X_train)
print(XX_train[0])
XX_train = np.array(X_train)
XX_train = X_train[1]
plt.scatter(X_train[1],XX_train)

norm.adapt(X_train)
X_val = norm(X_val)
X_val = np.array(X_val)

In [ ]:
# keras tuner for LSTM model


def build_model(hp):
  
  learning_rate = hp.Float("lr", min_value=1e-6, max_value=1e-3, sampling="log")       # learning rate search range
  layer_u = hp.Int("lu", min_value=10, max_value=32, step=4)                           # layer units search range
  kernel_r = hp.Float("kr", min_value=1e-10, max_value=1e-5, sampling="log")           # kernel regularization search range
  acti_f = hp.Choice("af", ['selu', 'tanh', 'relu', 'leaky_relu'])                     # activation function search range
  weight_d = hp.Float("wd", min_value=1e-10, max_value=0.0009, sampling="log")         # weight decay search range
  
# model structure
  model = tf.keras.Sequential([
      tf.keras.layers.LSTM(units = 34, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=0.00000195)),
      tf.keras.layers.LSTM(units = 26, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
      tf.keras.layers.LSTM(units = layer_u, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
      tf.keras.layers.LSTM(units = layer_u, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
      tf.keras.layers.LSTM(units = layer_u, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
      tf.keras.layers.LSTM(units = layer_u, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
      tf.keras.layers.LSTM(units = 30, activation = acti_f, return_sequences= False, kernel_regularizer=tf.keras.regularizers.L2(l2=0.00000195)),
      tf.keras.layers.Dense(units = 3, activation = 'softmax'),
  ])

# another model structure was used for experimenting
  # model = tf.keras.Sequential([
  #     tf.keras.layers.LSTM(units = 52, activation = acti_f, return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r)),
  #     tf.keras.layers.LSTM(units = layer_2, activation = acti_f, return_sequences= True, 
  #             kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.1, maxval=0.1)),
  #     tf.keras.layers.LSTM(units = layer_3, activation = acti_f, return_sequences= False, 
  #             kernel_regularizer=tf.keras.regularizers.L2(l2=kernel_r),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.1, maxval=0.1)),
  #     tf.keras.layers.Dense(units = 3, activation = 'softmax'),
  # ])


# the dense model structure experimented with
  # model = tf.keras.Sequential([ 
  #     tf.keras.layers.Dense(units = 330, activation = 'relu', batch_input_shape = (64,33)),    
  #     tf.keras.layers.Dense(units = 300, activation = 'relu', kernel_regularizer='l2'),
  #     tf.keras.layers.Dropout(0.2),                         
  #     tf.keras.layers.Dense(units = 250, activation = 'relu', kernel_regularizer='l2'),                         
  #     tf.keras.layers.Dropout(0.2),                               
  #     tf.keras.layers.Dense(units = 220, activation = 'relu', kernel_regularizer='l2'),   
  #     tf.keras.layers.Dropout(0.2),                                                
  #     tf.keras.layers.Dense(units = 200, activation = 'relu', kernel_regularizer='l2'), 
  #     tf.keras.layers.Dropout(0.2),                                                       
  #     tf.keras.layers.Dense(units = 150, activation = 'relu', kernel_regularizer='l2'), 
  #     tf.keras.layers.Dropout(0.2),                                                       
  #     tf.keras.layers.Dense(units = 10, activation = 'relu', kernel_regularizer='l2'),                         
  #     tf.keras.layers.Dense(units = 3, activation = 'softmax'),
  # ])



# Compiling the model
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer= tf.keras.optimizers.Adam(learning_rate = learning_rate, global_clipnorm=1, amsgrad = True, weight_decay=weight_d),
              metrics = [tf.keras.metrics.CategoricalCrossentropy(), tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.F1Score()])

  return model 

build_model(keras_tuner.HyperParameters())

tuner = keras_tuner.RandomSearch(                               # tuner configurations
    hypermodel=build_model,
    max_trials=15,
    objective=keras_tuner.Objective('val_loss', 'min'),
    executions_per_trial=1,
    overwrite=True,
    directory="/home/samer/Desktop/Big data Small Data/BDSD/Minor_project/emotion_estimation/",
    project_name="Emotion_estimation_tuning",
)

tuner.search_space_summary()

tuner.search(x=X_train, y=Y_train, validation_data = (X_val,y_val), epochs=100, batch_size = 150)

tuner.results_summary()



EStop = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5,restore_best_weights=True)


In [ ]:
# LSTM model fitting

# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units = 33, activation = 'relu', return_sequences= True),
#     tf.keras.layers.LSTM(units = 12, activation = 'relu', return_sequences= False),
#     tf.keras.layers.Dense(units = 3, activation = 'softmax'),
# ])

# the dense model trianed to compare the performance with the LSTM model
# model = tf.keras.Sequential([ 
#     tf.keras.layers.Dense(units = 330, activation = 'relu', batch_input_shape = (64,33)),    
#     tf.keras.layers.Dense(units = 300, activation = 'relu', kernel_regularizer='l2'),
#     tf.keras.layers.Dropout(0.2),                         
#     tf.keras.layers.Dense(units = 250, activation = 'relu', kernel_regularizer='l2'),                         
#     tf.keras.layers.Dropout(0.2),                               
#     tf.keras.layers.Dense(units = 220, activation = 'relu', kernel_regularizer='l2'),   
#     tf.keras.layers.Dropout(0.2),                                                
#     tf.keras.layers.Dense(units = 200, activation = 'relu', kernel_regularizer='l2'), 
#     tf.keras.layers.Dropout(0.2),                                                       
#     tf.keras.layers.Dense(units = 150, activation = 'relu', kernel_regularizer='l2'), 
#     tf.keras.layers.Dropout(0.2),                                                       
#     tf.keras.layers.Dense(units = 10, activation = 'relu', kernel_regularizer='l2'),                         
#     tf.keras.layers.Dense(units = 3, activation = 'softmax'),
# ])


# LSTM model final structure experimented with
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units = 34, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 26, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 30, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 30, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 30, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 30, activation = 'selu', return_sequences= True, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.LSTM(units = 30, activation = 'selu', return_sequences= False, kernel_regularizer=tf.keras.regularizers.L2(l2=2.256393134751847e-06)),
    tf.keras.layers.Dense(units = 3, activation = 'softmax'),
])

# # Compiling 
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
#             optimizer= tf.keras.optimizers.Adam(learning_rate = 1.0955e-06, clipnorm=1),
#             metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.F1Score(), tf.keras.metrics.CategoricalCrossentropy()])


# model = tf.keras.saving.load_model("/ckpt/checkpoint.model.keras")            # load the model from checkpoint to be trained


model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer= tf.keras.optimizers.AdamW(learning_rate = 5.6761842602901e-05, global_clipnorm=1, amsgrad = True, weight_decay=5.468661421085422e-05),
            metrics = [tf.keras.metrics.CategoricalCrossentropy(),tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.F1Score()])

# save model checkpoints
checkpoint_filepath = 'ckpt/epoch:{epoch:02d}-val_loss:{val_loss:.4f}.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(                  # model checkpoint callback
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose = 0)

# earlystopping callback
EStop = tf.keras.callbacks.EarlyStopping(monitor="categorical_crossentropy", mode = 'min', min_delta= 0.0001, patience=500, restore_best_weights=True, verbose=0)

# assigning weights for the classes (untuned)
weight_for_0 = 1
weight_for_1 = 1
weight_for_2 = 1

class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

# # Fitting the RNN to the Training set
model.fit(x=X_train, y=Y_train, validation_data = (X_val,y_val) ,epochs = 4000, batch_size=150, verbose=2, class_weight=class_weight, callbacks=[model_checkpoint_callback, EStop])
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


In [ ]:
# evaluate
model = tf.keras.models.load_model('/home/samer/Desktop/Big data Small Data/BDSD/Minor_project/BDSD_Minor_Project/trained_models/epoch4437val_loss0.6506.keras')            # load the model to be evaluated
tf.keras.utils.plot_model(model,                      # plot the model
    show_dtype=True,
    show_layer_names=True,
    expand_nested=True,
    show_layer_activations=True,
    show_trainable=True, show_shapes=True, rankdir="LR", to_file='foto.png')

test_blend_set = []
test_labels_set = []
test_index_set = []
with open("blends_test_index.csv", mode= "r") as test_data:             # load the test set
  csvFile = csv.reader(test_data)
  next(csvFile)
  for lines in csvFile:
      test_blend_set.append(lines[0:52])
      test_labels_set.append(lines[52])
      test_index_set.append(lines[53])
test_blend_set = np.array(test_blend_set, dtype=np.float64)
test_labels_set = np.array(test_labels_set, dtype=np.float64).astype('int')
test_labels_set = tf.keras.utils.to_categorical(test_labels_set, num_classes=3)
X_test = np.reshape(test_blend_set, (1646, 52,1))

model.evaluate(X_test,test_labels_set)                          # evaluate the model using the test set


model.save('LSTM_model_full_data_acc:63_f1:55.keras')

In [ ]:
# predict test classes to beused in further analysis

# new_model = tf.keras.models.load_model('LSTM_model_73%_test_acc')
predictions = []
for inst in X_test:
    inst = np.array(inst, dtype=np.float64)
    inst = np.reshape(inst, (1,52,1))

    y_pred = model.predict(inst, verbose=0)
    y_pred = np.argmax(y_pred)
    predictions.append(y_pred)


In [ ]:
# confusion matrix and error counting

errors_count = {"0":0, "1":0, "2":0}
ground_truth = []
errors = []
for i in test_labels_set:
    i = np.argmax(i)
    ground_truth.append(i)
for j in range(len(test_blend_set)):
    if predictions[j] != ground_truth[j]:
        errors.append(test_index_set[j])             # creat a list of the indices of the misclassified images in the testset
cm=tf.math.confusion_matrix(ground_truth,predictions,num_classes=3,dtype=tf.dtypes.int32,)        # calculate the confusion matrix

for i in range(1646):
    if predictions[i]==ground_truth[i]:
        continue
    elif predictions[i] != ground_truth[i]:
        errors_count[str(ground_truth[i])] = errors_count[str(ground_truth[i])] + 1


In [ ]:
# visualize misclassified images from the test set

test_blend_set = []
with open("test_set_full_index.csv", mode= "r") as test_data:
  csvFile = csv.reader(test_data)
  next(csvFile)
  for lines in csvFile:
      test_blend_set.append(lines[0:52])
      # imageee = np.array(str(lines[1]).split(' ')).reshape(48, 48, 1).astype(np.uint8)
      # imageee = cv2.cvtColor(imageee, cv2.COLOR_GRAY2RGB)
      # plt.imshow(imageee)
      # plt.show()
for ind in errors:
      print(ind)
      imageee = np.array(str(test_blend_set[int(float(ind))][1]).split(' ')).reshape(48, 48, 1).astype(np.uint8)
      imageees = cv2.cvtColor(imageee, cv2.COLOR_GRAY2RGB)
      frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=imageees)
      detection_result = detector.detect(frame)
      annotated_image = draw_landmarks_on_image(frame.numpy_view(), detection_result)
      plt.imshow(annotated_image)
      plt.show()
      plt.imshow(imageees)
      plt.show()
